# Orchestrating Chats
## Sequential Chat

As a sequence of chats between two agents chained together by a mechanism called carryover, the sequential chat pattern is useful for complex tasks that can be broken down into interdependent sub-tasks.

In this pattern, a pair of agents start a two-agent chat, then the summary of the conversation becomes a carryover for the next two-agent chat. The next chat passes the carryover to the carryover parameter of the context to generate its initial message.

Carryover accumulates as the conversation moves forward, so each subsequent chat starts with all the carryovers from previous chats.

The figure above shows distinct recipient agents for all the chats, however, the recipient agents in the sequence are allowed to repeat.

To illustrate this pattern, let’s create a sequential workflow to create our lesson plan for the fourth grade class.

A teacher agent will engage three agents sequentially:

1. A curriculum designer will select a topic given the subject from the teacher
2. A lesson planner will design the lesson with two iterations given feedback from the teacher
3. A formatter will format the lesson

In [1]:
from autogen import ConversableAgent

llm_config = {"api_type": "openai", "model": "gpt-4o-mini"}

# Three chats:
# 1. Teacher and Curriculum designer > summary is a topic for next chat
# 2. Teacher and Lesson planner (with 1 revision) > summary is lesson plan for next chat
# 3. Teacher and Formatter > summary is a formatted lesson plan

# Curriculum designer
curriculum_message = """You are a curriculum designer for a fourth grade class. Nominate an appropriate a topic for a lesson, based on the given subject."""

lesson_curriculum = ConversableAgent(
    name="curriculm_agent",
    llm_config=llm_config,
    system_message=curriculum_message,
)

# Lesson planner
planner_message = """You are a classroom lesson agent.
Given a topic, write a lesson plan for a fourth grade class in bullet points. Include the title, learning objectives, and script.
"""

lesson_planner = ConversableAgent(
    name="planner_agent",
    llm_config=llm_config,
    system_message=planner_message,
)

# Formatter
formatter_message = """You are a lesson plan formatter. Format the complete plan as follows:
<title>Lesson plan title</title>
<learning_objectives>Key learning objectives</learning_objectives>
<script>How to introduce the topic to the kids</script>
"""

lesson_formatter = ConversableAgent(
    name="formatter_agent",
    llm_config=llm_config,
    system_message=formatter_message,
)

# Teacher who initiates the chats
teacher_message = """You are a classroom teacher.
You decide topics for lessons and work with a lesson planner, you provide one round of feedback on their lesson plan.
Then you will work with a formatter to get a final output of the lesson plan.
"""

teacher = ConversableAgent(
    name="teacher_agent",
    llm_config=llm_config,
    system_message=teacher_message,
)

# Our sequential chat, each chat is a chat between the teacher and the recipient agent
# max_turns determines if there's back and forth between the teacher and the recipient
# max_turns = 1 means no back and forth
chat_results = teacher.initiate_chats(
    [
        {
            "recipient": lesson_curriculum,
            "message": "Let's create a science lesson, what's a good topic?",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": lesson_planner,
            "message": "Create a lesson plan.",
            "max_turns": 2, # One revision
            "summary_method": "last_msg",
        },
        {
            "recipient": lesson_formatter,
            "message": "Format the lesson plan.",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
    ]
)

# The result of `initiate_chats` is a list of chat results
# each chat result has a summary
print("\n\nCurriculum summary:\n", chat_results[0].summary)
print("\n\nLesson Planner summary:\n", chat_results[1].summary)
print("\n\nFormatter summary:\n", chat_results[2].summary)


********************************************************************************
Starting a new chat....

********************************************************************************
teacher_agent (to curriculm_agent):

Let's create a science lesson, what's a good topic?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
curriculm_agent (to teacher_agent):

A great topic for a fourth-grade science lesson would be "The Water Cycle." This topic can cover key concepts such as evaporation, condensation, precipitation, and collection. It provides an opportunity for hands-on activities, like creating a mini water cycle in a bag, and can lead to discussions about weather patterns and the importance of water conservation. The lesson can also incorporate visuals, experiments, and interactive components to engage the students effectively.

--------------------------------------------------------------------------------

**********

## Nested Chat

In [2]:
from autogen import ConversableAgent, GroupChatManager, GroupChat

llm_config = {
    "api_type": "openai",
    "model": "gpt-4o-mini",
}

# Curriculum Standards Agent
curriculum_agent = ConversableAgent(
    name="Curriculum_Agent",
    system_message="""You are a curriculum standards expert for fourth grade education.
    When given a topic, you provide relevant grade-level standards and learning objectives.
    Format every response as:
    STANDARDS:
    - [Standard 1]
    - [Standard 2]
    OBJECTIVES:
    - By the end of this lesson, students will be able to [objective 1]
    - By the end of this lesson, students will be able to [objective 2]""",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# Lesson Planner Agent
lesson_planner_agent = ConversableAgent(
    name="Lesson_Planner_Agent",
    system_message="""You are a lesson planning specialist.
    Given standards and objectives, you create detailed lesson plans including:
    - Opening/Hook (5-10 minutes)
    - Main Activity (20-30 minutes)
    - Practice Activity (15-20 minutes)
    - Assessment/Closure (5-10 minutes)
    Format as a structured lesson plan with clear timing and materials needed.""",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# Lesson Reviewer Agent
lesson_reviewer_agent = ConversableAgent(
    name="Lesson_Reviewer_Agent",
    system_message="""You are a lesson plan reviewer who ensures:
    1. Age-appropriate content and activities
    2. Alignment with provided standards
    3. Realistic timing
    4. Clear instructions
    5. Differentiation opportunities
    Provide specific feedback in these areas and suggest improvements if needed.""",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# Lead Teacher Agent
lead_teacher_agent = ConversableAgent(
    name="Lead_Teacher_Agent",
    system_message="""You are an experienced fourth grade teacher who oversees the lesson planning process.
    Your role is to:
    1. Initiate the planning process with a clear topic
    2. Review and integrate feedback from other agents
    3. Ensure the final lesson plan is practical and engaging
    4. Make final adjustments based on classroom experience""",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# Create the group chat for collaborative lesson planning
planning_chat = GroupChat(
    agents=[curriculum_agent, lesson_planner_agent, lesson_reviewer_agent],
    messages=[],
    max_round=4,
    send_introductions=True,
)

planning_manager = GroupChatManager(
    groupchat=planning_chat,
    llm_config=llm_config,
)

# Formatter of the final lesson plan to a standard format
formatter_message = """You are a lesson plan formatter. Format the complete plan as follows:
<title>Lesson plan title</title>
<standards>Standards covered</standards>
<learning_objectives>Key learning objectives</learning_objectives>
<materials>Materials required</materials>
<activities>Lesson plan activities</activities>
<assessment>Assessment details</assessment>
"""

lesson_formatter = ConversableAgent(
    name="formatter_agent",
    llm_config=llm_config,
    system_message=formatter_message,
)

# Create nested chats configuration
nested_chats = [
    {
        # The first internal chat determines the standards and objectives
        # A round of revisions is supported with max_turns = 2
        "recipient": curriculum_agent,
        "message": lambda recipient, messages, sender, config: f"Please provide fourth grade standards and objectives for the topic: {messages[-1]['content']}",
        "max_turns": 2,
        "summary_method": "last_msg",
    },
    {
        # A group chat follows, where the lesson plan is created
        "recipient": planning_manager,
        "message": "Based on these standards and objectives, create a detailed lesson plan.",
        "max_turns": 1,
        "summary_method": "last_msg",
    },
    {
        # Finally, a two-agent chat formats the lesson plan
        # The result of this will be the lead_teacher_agent's reply
        "recipient": lesson_formatter,
        "message": "Format the lesson plan.",
        "max_turns": 1,
        "summary_method": "last_msg",
    }
]

# Register nested chats with the lead teacher
lead_teacher_agent.register_nested_chats(
    chat_queue=nested_chats,
    trigger=lambda sender: sender not in [curriculum_agent, planning_manager, lesson_reviewer_agent],
)

# A human-in-the-loop agent
human = ConversableAgent(
    name="human_agent",
    human_input_mode="ALWAYS"
)

# A two-agent chat between our human and the lead_teacher_agent
# to demonstrate the full workflow is within the one agent
result = lead_teacher_agent.initiate_chat(
    recipient=human,
    message="What topic would you like to get a lesson plan for?",
    max_turns=2
)

print("Final Lesson Plan:\n", result.summary)

Lead_Teacher_Agent (to human_agent):

What topic would you like to get a lesson plan for?

--------------------------------------------------------------------------------
human_agent (to Lead_Teacher_Agent):

The earth

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Lead_Teacher_Agent (to Curriculum_Agent):

Please provide fourth grade standards and objectives for the topic: The earth

--------------------------------------------------------------------------------
Curriculum_Agent (to Lead_Teacher_Agent):

STANDARDS:
- Next Generation Science Standards (NGSS) 4-ESS2-1: Make observations and measurements to identify materials based on their properties.
- NGSS 4-ESS3-2: Generate and compare multiple solutions to reduce the impacts of natural Earth processes on humans.